<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Pipeline de Datos de Contratación Pública
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>


## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Abril 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Pipeline de Datos de Contratación Pública**|
|**Descripción y alcance**|Script para la consulta de bases y exploración de bases en el DATAVAULT.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

## Carga de librerías y ambiente de trabajo

En esta sección se cargan las librerías necesarias para el desarrollo del script y se establecen los parámetros de trabajo.


In [1]:
import pyodbc 
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
### Paquetes usados para la exploración de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
ruta_DATA="/dropbox/Dropbox/CCE-projects/Data/"

In [4]:
os.listdir(ruta_DATA)

['SECOP_II_Contratos_Depurado.parquet',
 'TVEC_Ordenes_Items.parquet',
 'SECOP_II_Contratos.parquet',
 'SECOP_I_Contratos_V0',
 'TVEC_Proveedores.parquet',
 'SECOP_I_Procesos',
 'SECOP_I_Contratos_Depurado.parquet',
 'SECOP_I_Procesos2023-06-06',
 'SECOP_I_Procesos.parquet',
 'SECOP_I_Procesos2023-06-05',
 'TVEC_Entidades.parquet',
 'SECOP_I_Contratos',
 'SECOP_I_Procesos_Depurado_V0',
 'SECOP_II_Procesos.parquet',
 'TVEC_Ordenes.parquet']

Generación de base integrada de TVEC

In [5]:
DF_TVEC_Ordenes=pd.read_parquet(ruta_DATA+"TVEC_Ordenes.parquet")
DF_TVEC_Ordenes_Items=pd.read_parquet(ruta_DATA+"TVEC_Ordenes_Items.parquet")
DF_TVEC_Entidades=pd.read_parquet(ruta_DATA+"TVEC_Entidades.parquet")
DF_TVEC_Proveedores=pd.read_parquet(ruta_DATA+"TVEC_Proveedores.parquet")

In [6]:
Ordenes_Entidades_SIN_ID=DF_TVEC_Ordenes[DF_TVEC_Ordenes['ID_Entidad'].isna()]
Ordenes_Proveedores_SIN_ID=DF_TVEC_Ordenes[DF_TVEC_Ordenes['ID_Proveedor'].isna()]

Ordenes_Entidades_SIN_ID.to_excel("../../../Sample_Data/Processed/Ordenes_Entidades_TVEC_SIN_ID.xlsx")
Ordenes_Proveedores_SIN_ID.to_excel("../../../Sample_Data/Processed/Ordenes_Proveedores_TVEC_SIN_ID.xlsx")

Se excluyen las entidades y los proveedores sin ID, esta 

In [7]:
DF_TVEC_Ordenes.dropna(subset=['ID_Entidad'],inplace=True)
DF_TVEC_Ordenes.dropna(subset=['ID_Proveedor'],inplace=True)
DF_TVEC_Ordenes.shape

(93366, 25)

In [8]:
DF_TVEC_Ordenes['ID_Entidad']=DF_TVEC_Ordenes['ID_Entidad'].astype(int).astype(str)
DF_TVEC_Ordenes['ID_Proveedor']=DF_TVEC_Ordenes['ID_Proveedor'].astype(int).astype(str)

In [9]:
DF_TVEC_Entidades['ID']=DF_TVEC_Entidades['ID'].fillna(0).astype(int).astype(str)
DF_TVEC_Proveedores['ID']=DF_TVEC_Proveedores['ID'].fillna(0).astype(int).astype(str)

In [10]:
DF_TVEC_Ordenes.shape

(93366, 25)

Integramos las tablas:

- DF_TVEC_Ordenes
- DF_TVEC_Entidades
- DF_TVEC_Proveedores

Usamos el diagrama ER Siguiente

[![](https://mermaid.ink/img/pako:eNqNkMEKwjAQRH8l7Nn6AQFPtoIHUVA8BUrorhq0iWxTQZr-u2mtQfDicWeGN-x0UDkkkECcG31mXSsrRL4qD8diWW4ZyVIjQsiyEJJcWG9QYzSkUDCF5uv8Y4iFSJEoK_iDuWP3IELHv9RkRe5XbCTDDGriWhuMP3RDjwJ_oZoUDBTUfB3q-5jTrXf7p61Aem5pBu0dtafpa5AnfWuSWqDxjpNI47l5LzUO1r8AZh1p8A?type=png)](https://mermaid.live/edit#pako:eNqNkMEKwjAQRH8l7Nn6AQFPtoIHUVA8BUrorhq0iWxTQZr-u2mtQfDicWeGN-x0UDkkkECcG31mXSsrRL4qD8diWW4ZyVIjQsiyEJJcWG9QYzSkUDCF5uv8Y4iFSJEoK_iDuWP3IELHv9RkRe5XbCTDDGriWhuMP3RDjwJ_oZoUDBTUfB3q-5jTrXf7p61Aem5pBu0dtafpa5AnfWuSWqDxjpNI47l5LzUO1r8AZh1p8A)

In [11]:
# Primero, unimos DF_TVEC_Ordenes con DF_TVEC_Entidades
df_merge_Ordenes_Entidades = pd.merge(DF_TVEC_Ordenes, DF_TVEC_Entidades, left_on='ID_Entidad', right_on='ID', suffixes=('_Ordenes', '_Entidades'),how='left')

# Luego, unimos el DataFrame resultante con DF_TVEC_Proveedores
df_final_TVEC = pd.merge(df_merge_Ordenes_Entidades, DF_TVEC_Proveedores, left_on='ID_Proveedor', right_on='ID', suffixes=('', '_Proveedores'),how='left')

# Ahora df_final es la integración de las tres bases de datos


In [12]:
df_final_TVEC.columns

Index(['ID_Ordenes', 'ID_Entidad', 'Entidad', 'Solicitante', 'Fecha_Ordenes',
       'Fecha_vence', 'ID_Proveedor', 'Proveedor', 'Estado', 'Solicitud',
       'Items', 'Total', 'Agregacion', 'Cotizacion', 'Padre', 'Ciudad_Ordenes',
       'Categoria', 'RFQ', 'Paz', 'Proceso', 'UNSPSC', 'Contrato', 'email',
       'Supervisor', 'Version', 'ID_Entidades', 'Nombre', 'NIT', 'Obligada',
       'Orden', 'Rama', 'Sector', 'Departamento', 'Ciudad_Entidades',
       'Fecha_Entidades', 'Active', 'Fechatemporal', 'Telefono', 'ID',
       'Nombre_Proveedores', 'NombreComercial', 'NIT_Proveedores',
       'Estado_Proveedores', 'Contacto', 'email_Proveedores', 'Direccion',
       'Ciudad', 'Departamento_Proveedores', 'Agregacion_Proveedores',
       'ActividadEconomica', 'RegTributario', 'FechaCreacion'],
      dtype='object')

In [13]:
df_final_TVEC.shape
df_final_TVEC['URL']='https://www.colombiacompra.gov.co/content/tienda-virtual'

In [14]:
df_final_TVEC['Modalidad']='TVEC'
df_final_TVEC['Tipo de contrato']='TVEC'
df_final_TVEC['Objeto Contrato']=df_final_TVEC['Agregacion']
df_final_TVEC['Tipo de documento proveedor']='NIT'

In [15]:
os.listdir(ruta_DATA)

['SECOP_II_Contratos_Depurado.parquet',
 'TVEC_Ordenes_Items.parquet',
 'SECOP_II_Contratos.parquet',
 'SECOP_I_Contratos_V0',
 'TVEC_Proveedores.parquet',
 'SECOP_I_Procesos',
 'SECOP_I_Contratos_Depurado.parquet',
 'SECOP_I_Procesos2023-06-06',
 'SECOP_I_Procesos.parquet',
 'SECOP_I_Procesos2023-06-05',
 'TVEC_Entidades.parquet',
 'SECOP_I_Contratos',
 'SECOP_I_Procesos_Depurado_V0',
 'SECOP_II_Procesos.parquet',
 'TVEC_Ordenes.parquet']

In [16]:
# Leer el archivo Parquet y especificar que se ignore el error de tiempo fuera de rango
DF_SECOP_II = pd.read_parquet(ruta_DATA+"SECOP_II_Contratos_Depurado.parquet", engine='fastparquet')

DF_SECOP_II.shape


(2634056, 80)

In [17]:
List_DF_SECOP_I_Contratos=[]
for file in os.listdir(ruta_DATA+'SECOP_I_Procesos/'):
    df_temp=pd.read_parquet(ruta_DATA+'SECOP_I_Procesos/'+file, engine='fastparquet')
    List_DF_SECOP_I_Contratos.append(df_temp)

DF_SECOP_I_Contratos=pd.concat(List_DF_SECOP_I_Contratos)
DF_SECOP_I_Contratos.reset_index(inplace=True)
DF_SECOP_I_Contratos.shape

(10029332, 95)

Integración de las bases usando Homologa Columnas:

| SECOP I                 | TVEC                        | SECOP II                   | Unificado                |
|-------------------------|-----------------------------|----------------------------|--------------------------|
| Fuente                  | Fuente                      | Fuente                     | Fuente                   |
| ID_ADJUDICACION         | ID_Ordenes                  | ContractReference          | ID Contrato              |
| UID                     | ID_Ordenes                  | RequestReference           | ID Proceso               |
| CODI_ENTIDAD            | ID_Entidad                  | Código Entidad              | ID entidad Plataforma    |
| NOMB_ENTIDAD            | Entidad                     | Nombre Entidad              | Entidad                  |
| NIT_ENTIDAD             | NIT                         | NIT Entidad                 | NIT Entidad              |
| ORDEN_ENTIDAD           | Orden                       | Orden                      | Orden                    |
| NOM_TIPO_PROCESO        | Modalidad                   | TypeContract               | Modalidad                |
| ESTADO_PROCESO          | Estado                      | ContractState              | Estado                   |
| DETALLE_OBJETO_PROCESO  | Agregacion                  | Description                | Descripción proceso      |
| OBJETO_CONTRATO         | Objeto Contrato             | Objeto Contrato             | Objeto Contrato          |
| TIPO_CONTRATO           | Tipo de contrato            | TypeContract               | Tipo de contrato         |
| FECHA_FIRMA_CONTRATO    | FechaCreacion               | ContractStartDate          | Fecha de Firma           |
| ID_CLASE                | UNSPSC                      | DimMainCategoryCode        | UNSPSC                   |
| RAZON_SOCIAL_CONTRATISTA| Nombre_Proveedores          | Nombre Proveedor            | Nombre Proveedor         |
| NUMERO_ID_CONTRATISTA   | NIT_Proveedores             | Documento Proveedor         | Documento Proveedor      |
| TIPO_ID_CONTRATISTA     | Tipo de documento proveedor | TipoDocProveedor           | Tipo de documento proveedor |
| CUANTIA_CONTRATAR       | Total                       | ContractValue              | Valor del contrato       |
| DEPARTAMENTO_ENTIDAD    | Departamento                | Departamento Entidad        | Departamento Entidad     |
| MUNICIPIO_ENTIDAD       | Ciudad_Entidades            | Ciudad Entidad              | Ciudad Entidad           |
| DPTO_MUNI_CONTRATISTA   | Departamento_Proveedores    | Departamento Proveedor      | Departamento Proveedor   |
| DPTO_MUNI_CONTRATISTA   | Ciudad                      | Ciudad Proveedor            | Ciudad Proveedor         |
| FECHA_FIRMA_CONTRATO    | Fecha_Ordenes               | ContractStartDate          | Inicio de contrato       |
| FECHA_FIN_EJE_CONTRATO  | Fecha_vence                 | ContractEndDate            | Fin de contrato          |
| RUTA_SECOPI             | URL                         | URLProceso                 | URL                      |



In [18]:
HC=pd.read_excel('../../../Sample_Data/Auxiliar/Homologa_columnas.xlsx')
HC.columns

Index(['Column1', 'SECOP I', 'TVEC', 'SECOP II', 'Unificado', 'SECOP I DA',
       'SECOP II DA', 'TVEC DA', 'Observaciones'],
      dtype='object')

In [19]:
DF_SECOP_I_Contratos['Fuente']='SECOP I'
DF_SECOP_II['Fuente']='SECOP II'
df_final_TVEC['Fuente']='TVEC'

In [20]:
DF_SECOP_I_Contratos.dropna(subset=['ID_ADJUDICACION'],inplace=True)
DF_SECOP_I_Contratos['ID_ADJUDICACION']=DF_SECOP_I_Contratos['ID_ADJUDICACION'].astype('int64').astype('str')


In [21]:
new_SECOP_I=pd.DataFrame()
new_SECOP_II=pd.DataFrame()
new_TVEC=pd.DataFrame()
for ind_column in HC.index:
    new_SECOP_I[HC['Unificado'][ind_column]]=DF_SECOP_I_Contratos[HC['SECOP I'][ind_column]]
    new_SECOP_II[HC['Unificado'][ind_column]]=DF_SECOP_II[HC['SECOP II'][ind_column]]
    new_TVEC[HC['Unificado'][ind_column]]=df_final_TVEC[HC['TVEC'][ind_column]]

del DF_SECOP_I_Contratos
del DF_SECOP_II
del df_final_TVEC

In [22]:
new_SECOP_I['UNSPSC']=new_SECOP_I['UNSPSC'].astype('str').str.replace('.0','00')


In [23]:
new_SECOP_II['UNSPSC']=new_SECOP_II['UNSPSC'].astype('str').str.replace('V1.','')

In [24]:
new_TVEC['UNSPSC']=new_TVEC['UNSPSC'].fillna('No Definido').str.split(';').str[0]

In [25]:
## Concatena los tres DataFrames

DF_Consulta=pd.concat([new_SECOP_I,new_SECOP_II,new_TVEC])
DF_Consulta.reset_index(inplace=True)

In [26]:
DF_Consulta.shape

(11915267, 26)

In [27]:
## Estado de los contratos

# Diccionario de estandarización de estados contractuales
estado_contractual = {
    'En ejecución': 'En ejecución',
    'Modificado': 'Modificado',
    'Celebrado': 'Celebrado',
    'terminado': 'Terminado',
    'Liquidado': 'Liquidado',
    'Cerrado': 'Cerrado',
    'Activo': 'Activo',
    'Terminado sin Liquidar': 'Terminado sin Liquidar',
    'cedido': 'Cedido',
    'Suspendido': 'Suspendido',
    'Convocado': 'Convocado',
    'Adjudicado': 'Adjudicado'
}

# Corregir los estados contractuales utilizando el diccionario
DF_Consulta['Estado'] = DF_Consulta['Estado'].replace(estado_contractual)
DF_Consulta['Estado'].value_counts()


Estado
Celebrado                                      5856927
Liquidado                                      2991138
En ejecución                                   1015068
Modificado                                      604149
Terminado                                       352822
Terminado sin Liquidar                          339116
Activo                                          237278
Cerrado                                         179651
Borrador                                        118053
Emitida                                          61915
Cancelado                                        59884
Cerrada                                          28373
enviado Proveedor                                27604
Cedido                                           19120
En aprobación                                    17141
Suspendido                                        3140
Cancelada                                         3075
Convocado                                          617
Pro

In [28]:
### Ajustemos las entidades

# Carga de MDM de entidades

MDM_Entidades = pd.read_excel('../../../Sample_Data/Auxiliar/MDM_Total.xlsx')

## 

In [29]:
MDM_SII=MDM_Entidades.drop_duplicates(subset=['Codigo_Entidad_SECOP_II'], keep='first')
MDM_SI=MDM_Entidades.drop_duplicates(subset=['Codigo_Entidad_SECOP_I'], keep='first')

In [30]:
MDM_Entidades.columns

Index(['NIT_SECOP', 'Codigo_SIGEP', 'Nombre_Entidad', 'Orden_Entidad',
       'Rama_Entidad', 'Sector_Entidad', 'Codigo_Entidad_SECOP_II',
       'Codigo_Entidad_SECOP_I', 'Codigo_Entidad'],
      dtype='object')

In [31]:
### Agregamos el código de entidad y corregimos el nombre de la entidad usando el MDM segun la fuente

# Agregamos el código de entidad

DF_Consulta['NIT_MDM']=np.nan
DF_Consulta['Codigo Entidad']=np.nan
for ind in DF_Consulta.index:
        lista_error=[]
        if DF_Consulta['Fuente'][ind]=='SECOP I':
            cod_entidad=MDM_SI[MDM_SI['Codigo_Entidad_SECOP_I']==DF_Consulta['ID entidad Plataforma'][ind]]
            if cod_entidad.empty or  len(cod_entidad)>1:
                lista_error.append([DF_Consulta['Entidad'][ind],DF_Consulta['ID entidad Plataforma'][ind],'SECOP I'])
            else:
                DF_Consulta['Codigo Entidad'][ind]=cod_entidad['Codigo_Entidad'].values[0]
                DF_Consulta['NIT_MDM'][ind]=cod_entidad['NIT_SECOP'].values[0]
            
        elif DF_Consulta['Fuente'][ind]=='SECOP II':
            cod_entidad=MDM_SII[MDM_SII['Codigo_Entidad_SECOP_II']==DF_Consulta['ID entidad Plataforma'][ind]]
            if cod_entidad.empty or  len(cod_entidad)>1:
                lista_error.append([DF_Consulta['Entidad'][ind],DF_Consulta['ID entidad Plataforma'][ind],'SECOP II'])
            else:
                DF_Consulta['Codigo Entidad'][ind]=cod_entidad['Codigo_Entidad'].values[0]
                DF_Consulta['NIT_MDM'][ind]=cod_entidad['NIT_SECOP'].values[0]
        else:
            cod_entidad=MDM_Entidades[MDM_Entidades['Nombre_Entidad']==DF_Consulta['Entidad'][ind]]
            if cod_entidad.empty or  len(cod_entidad)>1:
                lista_error.append([DF_Consulta['Entidad'][ind],DF_Consulta['ID entidad Plataforma'][ind],'TVEC'])
            else:
                DF_Consulta['Codigo Entidad'][ind]=cod_entidad['Codigo_Entidad'].values[0]
                DF_Consulta['NIT_MDM'][ind]=cod_entidad['NIT_SECOP'].values[0]
if len(lista_error)>0:
        Error_MDM=pd.DataFrame(lista_error,columns=['Entidad','ID entidad Plataforma','Fuente'])
        Error_MDM.to_excel('../../../Sample_Data/Processed/Errores/MDM_Total.xlsx',index=False) 


In [32]:
DF_Consulta['NIT_MDM'].isna().sum()


396363

In [33]:
DF_Consulta['Entidad']=DF_Consulta['Entidad'].str.upper()

In [34]:
## Exploración de los datos
# Vemos el tamaño del dataset
print("El dataset tiene", DF_Consulta.shape[0], "filas y", DF_Consulta.shape[1], "columnas")

# Vemos los tipos de datos
print(DF_Consulta.dtypes)

#Extraemos para este reporte las URL de los contratos

#DF_Consulta.drop('URL', axis=1, inplace=True)

El dataset tiene 11915267 filas y 28 columnas
index                            int64
Fuente                          object
ID Contrato                     object
ID Proceso                      object
ID entidad Plataforma           object
Entidad                         object
NIT Entidad                     object
Orden                           object
Modalidad                       object
Estado                          object
Descripción proceso             object
Objeto Contrato                 object
Tipo de contrato                object
Fecha de Firma                  object
UNSPSC                          object
Nombre Proveedor                object
Documento Proveedor             object
Tipo de documento proveedor     object
Valor del contrato             float64
Departamento Entidad            object
Ciudad Entidad                  object
Departamento Proveedor          object
Ciudad Proveedor                object
Inicio de contrato              object
Fin de contrato   

In [35]:
DF_Consulta['NIT_MDM'].isna().sum()

396363

In [36]:
### Arreglamos el orden
ordenes = {
    'Territorial departamental descentralizado': 'Territorial',
    'Territorial distrital municipal nivel 6': 'Territorial',
    'Territorial': 'Territorial',
    'Nacional': 'Nacional',
    'Territorial distrital municipal nivel 2': 'Territorial',
    'Territorial departamental centralizado': 'Territorial',
    'Nacional descentralizado': 'Nacional',
    'Territorial distrital municipal nivel 1': 'Territorial',
    'Nacional centralizado': 'Nacional',
    'Distrito capital': 'Territorial',
    'Territorial distrital municipal nivel 5': 'Territorial',
    'Territorial distrital municipal nivel 4': 'Territorial',
    'Territorial distrital municipal nivel 3': 'Territorial',
    'Corporación autónoma': 'Territorial',
    'Area metropolitana': 'Territorial',
    '': 'No definido',
    'No definido': 'No definido'
}
DF_Consulta['Orden']=DF_Consulta['Orden'].str.capitalize().replace(ordenes).fillna('No definido')
DF_Consulta['Orden'].value_counts(dropna=False)

Orden
Territorial    9719241
Nacional       2175854
No definido      20172
Name: count, dtype: int64

In [37]:
diccionario_modalidades = {
    'Asociación Público Privada': 'Modalidad Competitiva',
    'Régimen Especial': 'Modalidad Especial',
    'Iniciativa Privada sin recursos públicos': 'Modalidad Especial',
    'Contratación Directa (Ley 1150 de 2007)': 'Modalidad Directa',
    'Contratación Directa Menor Cuantía': 'Modalidad Directa',
    'Contratos y convenios con más de dos partes': 'Modalidad Directa',
    'Otras Formas de Contratación Directa': 'Modalidad Directa',
    'Concurso de diseño Arquitectónico': 'Modalidad Competitiva',
    'Concurso de Méritos Abierto': 'Modalidad Competitiva',
    'Concurso de Méritos con Lista Corta': 'Modalidad Competitiva',
    'Concurso de Méritos con Lista Multiusos': 'Modalidad Competitiva',
    'Licitación Pública': 'Modalidad Competitiva',
    'Licitación obra pública': 'Modalidad Competitiva',
    'Licitación Pública Nacional (BM)': 'Modalidad Competitiva',
    'Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)': 'Modalidad Competitiva',
    'Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007': 'Modalidad Competitiva',
    'Selección Abreviada servicios de Salud': 'Modalidad Competitiva',
    'Subasta': 'Modalidad Competitiva',
    'Contratación Mínima Cuantía': 'Modalidad Competitiva',
    'Invitación ofertas cooperativas o asociaciones de entidades territoriales': 'Modalidad Competitiva',
    'Lista Multiusos': 'Modalidad Competitiva',
    'Llamado a presentar expresiones de interés': 'Modalidad Competitiva',
    'Solicitud de información a los Proveedores': 'Modalidad Competitiva',
    'Contratación régimen especial': 'Modalidad Especial',
    'Contratación régimen especial (con ofertas)': 'Modalidad Especial',
    'Enajenación de bienes con sobre cerrado': 'Modalidad Competitiva',
    'Enajenación de bienes con subasta': 'Modalidad Competitiva',
    'Contratación directa': 'Modalidad Directa',
    'Selección Abreviada de Menor Cuantía': 'Modalidad Competitiva',
    'Mínima cuantía': 'Modalidad Competitiva',
    'Seleccion Abreviada Menor Cuantia Sin Manifestacion Interes': 'Modalidad Competitiva',
    'Licitación pública': 'Modalidad Competitiva',
    'Selección abreviada subasta inversa': 'Modalidad Competitiva',
    'Concurso de méritos abierto': 'Modalidad Competitiva',
    'Contratación Directa (con ofertas)': 'Modalidad Directa',
    'Licitación Pública Acuerdo Marco de Precios': 'Modalidad Competitiva',
    'Licitación pública Obra Publica': 'Modalidad Competitiva',
    'CCE-20-Concurso_Meritos_Sin_Lista_Corta_1Sobre': 'Modalidad Competitiva',
    'Concurso de méritos con precalificación': 'Modalidad Competitiva',
    'CCE-19-Concurso_Meritos_Con_Lista_Corta_1Sobre': 'Modalidad Competitiva',
    'Prestación de servicios': 'Modalidad Directa',
    'DecreeLaw092/2017': 'Otro',
    'Suministros': 'Otro',
    'Compraventa': 'Otro',
    'Arrendamiento de inmuebles': 'Otro',
    'Obra': 'Otro',
    'Comodato': 'Otro',
    'Interventoría': 'Otro',
    'No Especificado': 'Otro',
    'Consultoría': 'Otro',
    'Seguros': 'Otro',
    'Acuerdo Marco de Precios': 'Otro',
    'Arrendamiento de muebles': 'Otro',
    'Servicios financieros': 'Otro',
    'Emprestito': 'Otro',
    'Venta muebles': 'Otro',
    'Comisión': 'Otro',
    'Negocio fiduciario': 'Otro',
    'Concesión': 'Otro',
    'Venta inmuebles': 'Otro',
    'Acuerdo de cooperación': 'Otro'
}

DF_Consulta['Modalidad Corta']=DF_Consulta['Modalidad'].replace(diccionario_modalidades).fillna('No definido')
DF_Consulta['Modalidad Corta'].value_counts(dropna=False)

Modalidad Corta
Modalidad Directa        6900554
Modalidad Especial       3347746
Modalidad Competitiva    1176989
Otro                      396612
TVEC                       93366
Name: count, dtype: int64

In [38]:
### Variables de texto largo, las pasamos a mayúsculas
DF_Consulta['Descripción proceso']=DF_Consulta['Descripción proceso'].str.upper()
DF_Consulta['Objeto Contrato']=DF_Consulta['Objeto Contrato'].str.upper()


In [39]:
### Arreglamos Tipo de Contrato
arreglos_tipo_contrato={
    'Prestación de servicios': 'Prestación de Servicios',
    'Suministro': 'Suministros',
    'Suministros': 'Suministros',
    'Otro': 'Otros',
    'Compraventa': 'Compraventa',
    'Decreelaw092/2017': 'Otros',
    'Otro tipo de contrato': 'Otros',
    'Arrendamiento de inmuebles': 'Arrendamiento',
    'Obra': 'Obras',
    'Interventoría': 'Interventoría',
    'Comodato': 'Arrendamiento',
    'Consultoría': 'Servicios',
    'Arrendamiento': 'Arrendamiento',
    'Acuerdo marco de precios': 'Acuerdo Marco de Precios',
    'Concesión': 'Concesión',
    'Arrendamiento de muebles': 'Arrendamiento',
    'Seguros': 'Seguros',
    'Servicios financieros': 'Servicios Financieros',
    'Venta muebles': 'Compraventa',
    'Comisión': 'Comisión',
    'Asociación público privada': 'Asociación Público Privada',
    'Acuerdo marco': 'Acuerdo Marco de Precios',
    'Emprestito': 'Emprestito',
    'Fiducia': 'Fiducia',
    'No definido': 'No definido',
}


DF_Consulta['Tipo de contrato']=DF_Consulta['Tipo de contrato'].str.capitalize().replace(arreglos_tipo_contrato).fillna('No definido')
DF_Consulta['Tipo de contrato'].value_counts(dropna=False)

Tipo de contrato
Prestación de Servicios       9035180
Suministros                   1179729
Otros                          606907
Compraventa                    405271
Obras                          296540
Arrendamiento                  141982
Tvec                            93366
Servicios                       67411
Interventoría                   63429
No especificado                  6506
Acuerdo Marco de Precios         4682
Seguros                          4265
Crédito                          3059
Concesión                        3040
Fiducia                          1346
Asociación Público Privada        587
Servicios Financieros             519
No definido                       422
Emprestito                        378
Agregación de demanda             274
Comisión                          165
Negocio fiduciario                133
Venta inmuebles                    54
Acuerdo de cooperación             22
Name: count, dtype: int64

In [40]:
DF_Consulta['Fecha de Firma']=pd.to_datetime(DF_Consulta['Fecha de Firma'], format='%d/%m/%Y', errors='coerce')
DF_Consulta['Inicio de contrato']=pd.to_datetime(DF_Consulta['Inicio de contrato'], format='%d/%m/%Y', errors='coerce')
DF_Consulta['Fin de contrato']=pd.to_datetime(DF_Consulta['Fin de contrato'], format='%d/%m/%Y', errors='coerce')

In [42]:
import re

In [43]:
DF_Consulta['UNSPSC']=DF_Consulta['UNSPSC'].astype(str)
codigos_limpios = []
alertas_unspc = []
for codigo in DF_Consulta['UNSPSC']:
    codigo_limpio = re.sub('[^0-9]', '', codigo)
    
    if len(codigo_limpio) ==6:
        codigos_limpios.append(codigo_limpio+'00')
        alertas_unspc.append(False)
    elif len(codigo_limpio) ==9:
        codigos_limpios.append(codigo_limpio[1:])
        alertas_unspc.append(False)
    elif len(codigo_limpio) !=8:
        codigos_limpios.append(codigo_limpio)
        alertas_unspc.append(True)
    else:
        codigos_limpios.append(codigo_limpio)
        alertas_unspc.append(False)

DF_Consulta['UNSPSC']=codigos_limpios
DF_Consulta['Alerta UNSPSC']=alertas_unspc

In [44]:
DF_Consulta['Alerta UNSPSC'].value_counts(dropna=False)

Alerta UNSPSC
False    11355616
True       559651
Name: count, dtype: int64

In [45]:
### Proveedores

DF_Consulta['Nombre Proveedor']=DF_Consulta['Nombre Proveedor'].str.upper()
DF_Consulta['Documento Proveedor']=DF_Consulta['Documento Proveedor'].str.upper()
DF_Consulta['Tipo de documento proveedor']=DF_Consulta['Tipo de documento proveedor'].str.upper()

In [46]:
DF_Consulta.shape

(11915267, 30)

In [47]:
DF_Consulta['Fuente'].value_counts(dropna=False)

Fuente
SECOP I     9187845
SECOP II    2634056
TVEC          93366
Name: count, dtype: int64

In [48]:
DF_Consulta['Valor del contrato']=DF_Consulta['Valor del contrato'].astype(int)

In [49]:
DF_Consulta['Valor del contrato'].describe()

count    1.191527e+07
mean    -1.303452e+12
std      7.942238e+15
min     -9.223372e+18
25%      4.400000e+06
50%      1.100000e+07
75%      2.548438e+07
max      6.004813e+18
Name: Valor del contrato, dtype: float64

In [50]:
reemplazos_departamento = {
    'Bogotá DC': 'Bogotá D.C.',
    'Bogotá, D.C.': 'Bogotá D.C.',
    'Distrito Capital de Bogotá': 'Bogotá D.C.',
    'Bogotá D.C.': 'Bogotá D.C.',
    'Bogotá D.C.': 'Bogotá D.C.',
    'San Andrés Providencia y Santa Catalina': 'San Andrés, Providencia y Santa Catalina',
    'San Andres': 'San Andrés, Providencia y Santa Catalina',
    'NORTE DE SANTANDER': 'Norte de Santander',
    'Norte De Santander': 'Norte de Santander',
    'Bolívar': 'Bolivar',
    'caldas': 'Caldas',
    'Caldas ': 'Caldas',
    'CAUCA': 'Cauca',
    'Valle del cauca': 'Valle del Cauca',
    'Cordoba': 'Córdoba',
    'cundinamarca': 'Cundinamarca',
    'LA GUAJIRA': 'La Guajira',
    'huila': 'Huila',
    'Huila ': 'Huila',
    'meta': 'Meta',
    'SANTANDER': 'Santander',
    'Santander ': 'Santander',
    'SUCRE': 'Sucre'
}

DF_Consulta['Departamento Entidad']=DF_Consulta['Departamento Entidad'].replace(reemplazos_departamento)
DF_Consulta['Departamento Entidad'].value_counts(dropna=False)

Departamento Entidad
Bogotá D.C.                                 2156377
Antioquia                                   1537668
No Definido                                  883046
Valle del Cauca                              804024
Cundinamarca                                 707835
Nariño                                       553931
Santander                                    531837
Boyacá                                       474433
Norte de Santander                           352024
Tolima                                       335700
Caldas                                       309727
Meta                                         307295
Huila                                        279060
Magdalena                                    255708
Bolivar                                      243347
Cauca                                        243053
Atlántico                                    221870
Casanare                                     217664
Quindío                                    

In [51]:
DF_Consulta['Departamento Entidad'].value_counts(dropna=False)

Departamento Entidad
Bogotá D.C.                                 2156377
Antioquia                                   1537668
No Definido                                  883046
Valle del Cauca                              804024
Cundinamarca                                 707835
Nariño                                       553931
Santander                                    531837
Boyacá                                       474433
Norte de Santander                           352024
Tolima                                       335700
Caldas                                       309727
Meta                                         307295
Huila                                        279060
Magdalena                                    255708
Bolivar                                      243347
Cauca                                        243053
Atlántico                                    221870
Casanare                                     217664
Quindío                                    

In [52]:
DF_Consulta['Ciudad Entidad']=DF_Consulta['Ciudad Entidad'].str.capitalize()

In [55]:
DF_Consulta[DF_Consulta['Fuente']=='SECOP II']['URL']

9187845     https://community.secop.gov.co/Public/Tenderin...
9187846     https://community.secop.gov.co/Public/Tenderin...
9187847     https://community.secop.gov.co/Public/Tenderin...
9187848     https://community.secop.gov.co/Public/Tenderin...
9187849     https://community.secop.gov.co/Public/Tenderin...
                                  ...                        
11821896    https://community.secop.gov.co/Public/Tenderin...
11821897    https://community.secop.gov.co/Public/Tenderin...
11821898    https://community.secop.gov.co/Public/Tenderin...
11821899    https://community.secop.gov.co/Public/Tenderin...
11821900    https://community.secop.gov.co/Public/Tenderin...
Name: URL, Length: 2634056, dtype: object

In [56]:
DF_Consulta.reset_index(drop=True, inplace=True)

In [ ]:
DF_Consulta.to_parquet('../../../Sample_Data/Processed/SECOP_LIMPIO_08_06.parquet', index=False,engine='fastparquet')

In [ ]:
import pandas as pd
import os

In [ ]:
DF_Consulta=pd.read_parquet('../../../Sample_Data/Processed/SECOP_LIMPIO_2020_2023.parquet')

In [ ]:
DF_Consulta=pd.read_excel('../../../Sample_Data/Processed/SECOP_LIMPIO_2023.xlsx')

In [ ]:
DF_Consulta[DF_Consulta['Fuente']=='SECOP II'].groupby('Año-Mes').agg({'Valor del contrato':'sum'})

,Valor del contrato
Año-Mes,
2023-01,129235903698772
2023-02,13593175257555
2023-03,149992538806700
2023-04,6625804806915
2023-05,5488785987149


In [ ]:
DF_Consulta.to_csv('../../../Sample_Data/Processed/SECOP_LIMPIO_2023.csv', index=False)

In [ ]:
from ydata_profiling import ProfileReport
### Reporte con pandas profiling
os.makedirs("../Reportes", exist_ok=True)
report=ProfileReport(DF_Consulta, title="Reporte de Exploración de Datos", 
                        minimal=True,dark_mode=True)

In [ ]:
report.to_file("../Reportes/Reporte de Exploración de Datos_2023.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Trabajo en datalake

En esta sección se establecen los parámetros de conexión al datalake y se cargan las librerías necesarias para la conexión y consulta de datos.

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# URL del punto final del servicio Blob y token SAS
url_blob = "https://adlindicadoresemae.blob.core.windows.net/"
token_sas = "?sv=2021-10-04&ss=btqf&srt=sco&st=2023-04-26T10%3A55%3A03Z&se=2023-07-27T10%3A55%3A00Z&sp=rwdxftlacup&sig=9%2FBvXPQRTnl4CUwGzrUpMFXbpu50nNRqix5pREicSiY%3D"

# Conexión a la cuenta de almacenamiento de Azure utilizando el token SAS
connect_str = f"BlobEndpoint={url_blob};SharedAccessSignature={token_sas}"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [ ]:
# Referencia al contenedor donde se encuentran los archivos
container_name_ent = "datosemae/bronze/Entidades"
container_client_ent = blob_service_client.get_container_client(container_name_ent)

# Nombre del archivo a leer
blob_name_ent_small = "Entidades_SECOP_II.parquet"
blob_name_ent_full = "Entidades_SECOP_II_FULL.parquet"